<a href="https://colab.research.google.com/github/FelipeTufaile/MixtureModels/blob/main/MixtureModels/tests/Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np